In [1]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Subset
import numpy as np
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

/home/jaeyeob/anaconda3/envs/lstm/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

# MNIST 데이터 불러오기
(X_train_full, y_train_full), (X_test_full, y_test_full) = mnist.load_data()

# 데이터 정규화 (0-255 값을 0-1 사이로)
X_train_full = X_train_full.astype('float32') / 255.0
X_test_full = X_test_full.astype('float32') / 255.0

#torch.transform / normalize // imagenet mean sd 

# 숫자 1과 7만 선택하는 마스크 생성
train_mask = np.isin(y_train_full, [1, 7])
X_train, y_train = X_train_full[train_mask], y_train_full[train_mask]

test_mask = np.isin(y_test_full, [1, 7])
X_test, y_test = X_test_full[test_mask], y_test_full[test_mask]

# 시드 고정 (예: 42로 고정)
np.random.seed(42)

# 2000개의 데이터를 무작위로 선택
num_samples = 200
indices = np.random.choice(len(X_train), num_samples, replace=False)
X_sampled, y_sampled = X_train[indices], y_train[indices]

# 2000개의 샘플에서 train/test 데이터 분할 (80% train, 20% test 비율로 나눔)
X_train, X_test, y_train, y_test = train_test_split(
    X_sampled, y_sampled, stratify=y_sampled, test_size=0.2, random_state=42
)

# 결과
print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

2024-11-20 00:41:34.803116: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-20 00:41:34.819266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-20 00:41:34.838291: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-20 00:41:34.844164: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-20 00:41:34.859258: I tensorflow/core/platform/cpu_feature_guar

Training set size: 160
Test set size: 40


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, average_precision_score
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

# 하이퍼파라미터 설정
batch_size = 16
learning_rate = 0.001
num_epochs = 10


# PyTorch Tensor로 변환
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # 채널 차원 추가
y_train = torch.tensor((y_train == 7).astype(np.float32))  # 1은 0으로, 7은 1로 변환
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor((y_test == 7).astype(np.float32))

# 데이터셋과 데이터 로더 정의
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# CNN 모델 정의
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# 모델 초기화
model = CNNModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 루프
for epoch in range(num_epochs):
    model.train()
    train_correct = 0
    train_total = 0
    for images, labels in train_loader:
        labels = labels.unsqueeze(1)  # BCELoss에 맞추어 차원 맞추기

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accuracy 계산
        predicted = (outputs > 0.5).float()
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)

    train_accuracy = 100 * train_correct / train_total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {train_accuracy:.2f}%')



/home/jaeyeob/anaconda3/envs/lstm/lib/python3.10/site-packages/torch/autograd/graph.py:825: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch [1/10], Loss: 0.3144, Accuracy: 73.12%
Epoch [2/10], Loss: 0.0595, Accuracy: 97.50%
Epoch [3/10], Loss: 0.1071, Accuracy: 96.25%
Epoch [4/10], Loss: 0.0036, Accuracy: 99.38%
Epoch [5/10], Loss: 0.0012, Accuracy: 99.38%
Epoch [6/10], Loss: 0.0031, Accuracy: 100.00%
Epoch [7/10], Loss: 0.0021, Accuracy: 100.00%
Epoch [8/10], Loss: 0.0004, Accuracy: 100.00%
Epoch [9/10], Loss: 0.0008, Accuracy: 100.00%
Epoch [10/10], Loss: 0.0002, Accuracy: 100.00%


In [4]:
# Print out the parameters and their shapes
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Parameter name: {name}")
        print(f"Shape: {param.shape}")
        print(f"Number of parameters: {param.numel()}")
        print("-" * 50)

# Total number of parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of trainable parameters: {total_params}")


Parameter name: conv1.weight
Shape: torch.Size([32, 1, 3, 3])
Number of parameters: 288
--------------------------------------------------
Parameter name: conv1.bias
Shape: torch.Size([32])
Number of parameters: 32
--------------------------------------------------
Parameter name: conv2.weight
Shape: torch.Size([64, 32, 3, 3])
Number of parameters: 18432
--------------------------------------------------
Parameter name: conv2.bias
Shape: torch.Size([64])
Number of parameters: 64
--------------------------------------------------
Parameter name: fc1.weight
Shape: torch.Size([128, 3136])
Number of parameters: 401408
--------------------------------------------------
Parameter name: fc1.bias
Shape: torch.Size([128])
Number of parameters: 128
--------------------------------------------------
Parameter name: fc2.weight
Shape: torch.Size([1, 128])
Number of parameters: 128
--------------------------------------------------
Parameter name: fc2.bias
Shape: torch.Size([1])
Number of parameters

In [4]:
# 테스트 평가
model.eval()
all_labels = []
all_preds = []
with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.unsqueeze(1)
        outputs = model(images)
        predicted = (outputs > 0.5).float()
        
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(outputs.cpu().numpy())

# 리스트를 numpy 배열로 변환
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# 이진 분류 결과 계산
test_accuracy = accuracy_score(all_labels, all_preds > 0.5)
test_roc_auc = roc_auc_score(all_labels, all_preds)
test_precision = precision_score(all_labels, all_preds > 0.5)
test_recall = recall_score(all_labels, all_preds > 0.5)
test_f1 = f1_score(all_labels, all_preds > 0.5)
test_auprc = average_precision_score(all_labels, all_preds)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"AUROC: {test_roc_auc:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(f"AUPRC: {test_auprc:.4f}")

Test Accuracy: 97.50%
AUROC: 1.0000
Precision: 0.9524
Recall: 1.0000
F1 Score: 0.9756
AUPRC: 1.0000
